# Convert .Rmd to .ipynb


In [ ]:
# install.packages("devtools")
# remove.packages("rlang")
# install.packages("rlang")
# devtools::install_github("mkearney/rmd2jupyter")
# install.packages("rmd2jupyter")
library("rmd2jupyter")

In [ ]:
# rmd to ipynb
rmd2jupyter("Preprocessing Data for Meta-Analyses.Rmd")

In [ ]:
install.packages("rmarkdown")

Installiere Paket nach 'C:/Users/4_20/AppData/Local/R/win-library/4.2'
(da 'lib' nicht spezifiziert)



Paket 'rmarkdown' erfolgreich ausgepackt und MD5 Summen abgeglichen

Die heruntergeladenen Binärpakete sind in 
	C:\Users\4_20\AppData\Local\Temp\Rtmp4GzrvA\downloaded_packages


In [ ]:
# test

# Preprocess data of raw covidence Export



In [ ]:
raw.df <- read.csv("final_data_export.csv")




## Install and load Dplyr


In [ ]:
# install.packages("dplyr")
library("dplyr")
# install.packages("sjmisc")
library("sjmisc")
# for data manipulation with dplyr see: https://www.youtube.com/watch?v=Gvhkp-Yw65U
# for splitting 2 values in 1 cell see: https://www.youtube.com/watch?v=DiY8EqZDwoI at 3:17 (e.g. if 2 scales for 1 outcome)
# for joining 2 data frames see:        https://www.youtube.com/watch?v=DiY8EqZDwoI at 11:57


In [ ]:
my.raw.df <- raw.df %>%
  filter(Reviewer.Name == "Robin Jacob", Study.design == "Passive RCT")


## Drop unimportant columns by name
### Remove column rages (first columns)


In [ ]:
my.df <- my.raw.df %>%
  select(-Reviewer.Name:-Further.Information.inserted.in.Extraction.Form.)  # "-" indicates deleting these columns



### Remove single column names and repeating names with ascending numbers (table headlines)


In [ ]:
cols.pop.char <- c()

for (time.point in 0:4){
  cols.pop.char <- append(cols.pop.char,
                          sprintf("Population.Characteristics..Time.Point.%d..",
                                  time.point
                                  )
                          )
}

cols.outc.t <- c()

for (outc in 1:7){
  for (t in 0:3){
    cols.outc.t <- append(cols.outc.t,
                          sprintf("Outcome.%d..Time.Point.%d..",
                                  outc, t
                                  )
                          )
  }
}

remove.cols <- c("Dates.of.Measuring.Time.Points.",
                 cols.pop.char[],
                 "Between.Measuring.Time.Points..Duration.",
                 "Intervention.and.Comparisons.",
                 "Definition.of.Outcomes.",
                 "Measures.of.Outcomes.",
                 cols.outc.t[]
                 )

my.df <- my.df[, ! names(my.df) %in% remove.cols]



## Create arrays and data frame lists from Covidence tables


In [ ]:
# install.packages("sjmisc")  # for str_contains()
library("sjmisc")



### Set basic parameters


In [ ]:
nm.placeholder <- -999  # placeholder for values marked as nm (not mentioned)

flag.x.s.r <- 2
  # x.s.r = exact value (0), mean s (1), or mean r (2)
  # flag.x.s.r = 2 --> include all
  # flag.x.s.r = 1 --> only include mean s and exact values
  # flag.x.s.r = 0 --> include only exact vaules

study.no <- length(my.df[,"Study.ID"])


### Functions
#### Checking for digits and characters


In [ ]:
# function returning TRUE if string contains no characters but digits instead
no.char.but.digit.inside <- function(x){
  !(str_contains(x, letters, logic = "OR") |
  str_contains(x, LETTERS, logic = "OR")) &
  grepl("\\d", x)
}

# function returning TRUE if string contains characters
char.inside <- function(x){
  str_contains(x, letters, logic = "OR") |
  str_contains(x, LETTERS, logic = "OR")
}



#### For mean values


In [ ]:
get.all.means <- function(value){
  if (grepl("mean r", value)){
    extracted.value <- as.double(sub(" mean r.*", "", value))
        # extracts anything before " mean r" as double                 
  } else if (grepl("mean s", value)){
    extracted.value <- as.double(sub(" mean s.*", "", value))
  } else if (grepl("mean", value)){
    extracted.value <- NA
    cat("value ", value, " not added because mean only")
    cat("\n")
  } else {
    extracted.value <- NA
    cat("unknown case: ", value)
    cat("\n")
  }
  extracted.value
}

get.mean.s.only <- function(value){
    if (grepl("mean s", value)){
    extracted.value <- as.double(sub(" mean s.*", "", value))
  } else if (grepl("mean r", value)){
    extracted.value <- NA
    cat("value ", value, " not added because mean r")
    cat("\n")
  } else if (grepl("mean", value)){
    extracted.value <- NA
    cat("value ", value, " not added because mean only")
    cat("\n")
  } else {
    extracted.value <- NA
    cat("unknown case: ", value)
    cat("\n")
  }
  extracted.value
}



#### convert value from nm, NA, digit, or Char


In [ ]:
convert.value <- function(value){
  if (is.na(value)){
    next
  }
  value <- as.character(value)
  if (value == "nm"){
    extracted.value <- nm.placeholder
  } else if (value == ""){
    extracted.value <- NA
  } else if (no.char.but.digit.inside(value)){
    extracted.value <- as.double(value)
  } else if (char.inside(value)){
    if (flag.x.s.r == 2){
      extracted.value <- get.all.means(value)
    } else if (flag.x.s.r == 1){
      extracted.value <- get.mean.s.only(value)
    } else if ((flag.x.s.r == 0) & !(grepl("mean r", value) | grepl("mean s", value))){
      extracted.value <- NA
      if (grepl("mean", value)){
        cat("value ", value, " not added because mean only")
        cat("\n")
      }
    }
  } else{
    cat("unknown case: ", value)
    cat("\n")
  }
  extracted.value
}



#### For arrays


In [ ]:
clean.and.shape.data.to.array <- function(start, end, dims, dimname.list){
  my.array <- array(rep(NA),
                    dim = dims,
                    dimnames = dimname.list
                    )
  my.data <- my.df %>%
    select(start:end)
  
  if (length(dims) == 6){
    for (study in 1:study.no){
      my.data.col.no = 1
      for (dim.4.elmnt in 1:dims[4]){
        for (mtrx in 1:dims[3]){
          known.case.flag <- FALSE
          for (row in 1:dims[1]){
            for (col in 1:dims[2]){
              value <- my.data[study, my.data.col.no]
              my.data.col.no <- my.data.col.no + 1
              if (grepl(",", value)){
                value.before.comma <- as.double(sub(",.*", "", value))
                  # extracts anything before "," as double
                my.array[row, col, mtrx, dim.4.elmnt, 1, study] <- convert.value(value.before.comma)
                
                value.after.comma <- as.double(sub(".*,", "", value))
                  # extracts anything after "," as double
                my.array[row, col, mtrx, dim.4.elmnt, 2, study] <- convert.value(value.after.comma)
              } else {
                my.array[row, col, mtrx, dim.4.elmnt, 1, study] <- as.double(value)
              }
            }
          }
        }
      }
    }
  }
  if (length(dims) == 5){
    for (study in 1:study.no){
      my.data.col.no = 1
      for (mtrx in 1:dims[3]){
        known.case.flag <- FALSE
        for (row in 1:dims[1]){
          for (col in 1:dims[2]){
            value <- my.data[study, my.data.col.no]
            my.data.col.no <- my.data.col.no + 1
# NA or nm
            if (is.na(value)){
              next
            }
            value <- as.character(value)
            if (value == "nm"){
              my.array[row, col, mtrx, 1, study] <- nm.placeholder
            } else if (value == ""){
              # values stay NA
# digits
            } else if (no.char.but.digit.inside(value)){
              if (grepl(",", value)){
                my.array[row, col, mtrx, 1, study] <- as.double(sub(",.*", "", value))
                  # extracts anything before "," as double
                my.array[row, col, mtrx, 2, study] <- as.double(sub(".*,", "", value))
                  # extracts anything after "," as double
              } else {
                my.array[row, col, mtrx, 1, study] <- as.double(value)
              }
# characters
            } else if (char.inside(value)){
              if (flag.x.s.r == 2){
                if (grepl(",", value)){
                  value.before.comma <- sub(",.*", "", value)
                  my.array[row, col, mtrx, 1, study] <- get.all.means(value.before.comma)
                    
                  value.after.comma <- sub(".*,", "", value)
                  my.array[row, col, mtrx, 2, study] <- get.all.means(value.after.comma)
                } else {
                  my.array[row, col, mtrx, 1, study] <- get.all.means(value)
                }
              } else if (flag.x.s.r == 1){
                if (grepl(",", value)){
                  value.before.comma <- sub(",.*", "", value)
                  my.array[row, col, mtrx, 1, study] <- get.mean.s.only(value.before.comma)
                    
                  value.after.comma <- sub(".*,", "", value)
                  my.array[row, col, mtrx, 2, study] <- get.mean.s.only(value.after.comma)
                } else {
                  my.array[row, col, mtrx, 1, study] <- get.mean.s.only(value)
                }
              } else if ((flag.x.s.r == 0) & !(grepl("mean r", value) | grepl("mean s", value))){
                if (grepl("mean", value)){
                  cat("value ", value, " not added because mean only")
                  cat("\n")
                }
              }
            } else {
              if (known.case.flag == FALSE){
                print("Unnown case appeared in:")
                print(my.df[study.no, "Study.ID"])
                print(sprintf("T = %d", t))
                print(my.array)
              }
              cat(sprintf("row %d, col %d\n\n", row, col))
              known.case.flag <- TRUE
            }
          }
        }
      }
    }
  }
  if (length(dims) <= 4){
    print("too low dim, use function for dataframe")
  }
  my.array
}


In [ ]:
# create data frames out of 2D Tables (with rows and cols swapped)
clean.data.to.df.list.swap <- function(start, end, dims, list.names, dimname.list){
  study.df <- data.frame(matrix(
    rep(NA),
    nrow = dims[1],
    ncol = dims[2],
  ))
  row.names(study.df) <- dimname.list[[1]]
  colnames(study.df) <- dimname.list[[2]]
  
  # create list of NA data frames of preferred shape
  study.df.list <- list()
  for (i in 1:length(list.names)){
    study.df.list <- append(study.df.list, list(study.df), 0)
  }
  names(study.df.list) <- list.names
  
  my.data <- my.df %>%
    select(start:end)
  
  # vector of my.data.col.no to swapped rows and cols of Covidence output
  my.data.col.no.swapped <- c()
  i <- 0
  j <- 1
  for (k in 1:dims[1]){
    i <- 0
    for (l in 1:dims[2]){
      my.data.col.no.swapped <- append(my.data.col.no.swapped, i * dims[1] + j)
      i <- i + 1
    }
    j <- j + 1 
  }
  # dims[1] = rows new, cols old
  # dims[2] = cols new, rows old
  
  # e.g., filling pattern for old 4 x 7 and new 7 x 4 data frame 
  ## 1, dims[1]+1, 2*dims[1]+1, 3*dims[1]+1, 4*dims[1]+1, 5*dims[1]+1, 6*dims[1]+1,
  ## 2, dims[1]+2, 2*dims[1]+2, 3*dims[1]+2, 4*dims[1]+2, 5*dims[1]+2, 6*dims[1]+2,
  ## x, x, x, x, x, x, x,
  ## x, x, x, x, x, x, x
  

  for (study in 1:study.no){
    m = 1
    for (row in 1:dims[1]){
      for (col in 1:dims[2]){
        value <- my.data[study, my.data.col.no.swapped[m]]
        m <- m + 1
        if (is.na(value)){
          next
        }
        value <- as.character(value)
        if (value == "nm"){
          study.df.list[[study]][row, col] <- nm.placeholder
        } else if (value == ""){
          # values stay NA
        } else if (no.char.but.digit.inside(value)){
          study.df.list[[study]][row, col] <- as.double(value)
        } else if (char.inside(value)){
          if (flag.x.s.r == 2){
            if (grepl("mean r", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean r.*", "", value))
                  # extracts anything before " mean r" as double                 
            } else if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if (flag.x.s.r == 1){
            if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean r", value)){
              # print(sprintf("mean r in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if ((flag.x.s.r == 0) & !(grepl("mean r", value) | grepl("mean s", value))){
            if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
              next
            }
            study.df.list[[study]][row, col] <- value
          } else {
            print("Unnown case appeared in:")
            print(my.df[study.no, "Study.ID"])
            cat("my.data.col.no: ", my.data.col.no.swapped[m])
            cat(sprintf("row %d, col %d", row, col))
            print(value)
            cat("\n")
          }
        } else {
          print("Unnown case appeared in:")
          print(my.df[study.no, "Study.ID"])
          cat("my.data.col.no: ", my.data.col.no.swapped[m])
          cat(sprintf("row %d, col %d\n\n", row, col))
        }
      }
    }
  }
  study.df.list
}



#### For data frame lists


In [ ]:
# create data frames out of 2D Tables
clean.data.to.df.list <- function(start, end, dims, list.names, dimname.list){
  study.df <- data.frame(matrix(
    rep(NA),
    nrow = dims[1],
    ncol = dims[2],
  ))
  row.names(study.df) <- dimname.list[[1]]
  colnames(study.df) <- dimname.list[[2]]
  
  # create list of NA data frames of preferred shape
  study.df.list <- list()
  for (i in 1:length(list.names)){
    study.df.list <- append(study.df.list, list(study.df), 0)
  }
  names(study.df.list) <- list.names
  
  my.data <- my.df %>%
    select(start:end)
  
  for (study in 1:study.no){
    my.data.col.no = 1
    for (row in 1:dims[1]){
      for (col in 1:dims[2]){
        value <- my.data[study, my.data.col.no]
        my.data.col.no <- my.data.col.no + 1
        if (is.na(value)){
          next
        }
        value <- as.character(value)
        if (value == "nm"){
          study.df.list[[study]][row, col] <- nm.placeholder
        } else if (value == ""){
          # values stay NA
        } else if (no.char.but.digit.inside(value)){
          study.df.list[[study]][row, col] <- as.double(value)
        } else if (char.inside(value)){
          if (flag.x.s.r == 2){
            if (grepl("mean r", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean r.*", "", value))
                  # extracts anything before " mean r" as double                 
            } else if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if (flag.x.s.r == 1){
            if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean r", value)){
              # print(sprintf("mean r in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if ((flag.x.s.r == 0) & !(grepl("mean r", value) | grepl("mean s", value))){
            if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
              next
            }
            study.df.list[[study]][row, col] <- value
          } else {
            print("Unnown case appeared in:")
            print(my.df[study.no, "Study.ID"])
            cat("my.data.col.no: ", my.data.col.no.swapped[m])
            cat(sprintf("row %d, col %d", row, col))
            print(value)
            cat("\n")
          }
        } else {
          print("Unnown case appeared in:")
          print(my.df[study.no, "Study.ID"])
          cat("my.data.col.no: ", my.data.col.no.swapped[m])
          cat(sprintf("row %d, col %d\n\n", row, col))
        }
      }
    }
  }
  study.df.list
}


In [ ]:
# create data frames out of 2D Tables (with rows and cols swapped)
clean.data.to.df.list.swap <- function(start, end, dims, list.names, dimname.list){
  study.df <- data.frame(matrix(
    rep(NA),
    nrow = dims[1],
    ncol = dims[2],
  ))
  row.names(study.df) <- dimname.list[[1]]
  colnames(study.df) <- dimname.list[[2]]
  
  # create list of NA data frames of preferred shape
  study.df.list <- list()
  for (i in 1:length(list.names)){
    study.df.list <- append(study.df.list, list(study.df), 0)
  }
  names(study.df.list) <- list.names
  
  my.data <- my.df %>%
    select(start:end)
  
  # vector of my.data.col.no to swapped rows and cols of Covidence output
  my.data.col.no.swapped <- c()
  i <- 0
  j <- 1
  for (k in 1:dims[1]){
    i <- 0
    for (l in 1:dims[2]){
      my.data.col.no.swapped <- append(my.data.col.no.swapped, i * dims[1] + j)
      i <- i + 1
    }
    j <- j + 1 
  }
  # dims[1] = rows new, cols old
  # dims[2] = cols new, rows old
  
  # e.g., filling pattern for old 4 x 7 and new 7 x 4 data frame 
  ## 1, dims[1]+1, 2*dims[1]+1, 3*dims[1]+1, 4*dims[1]+1, 5*dims[1]+1, 6*dims[1]+1,
  ## 2, dims[1]+2, 2*dims[1]+2, 3*dims[1]+2, 4*dims[1]+2, 5*dims[1]+2, 6*dims[1]+2,
  ## x, x, x, x, x, x, x,
  ## x, x, x, x, x, x, x
  

  for (study in 1:study.no){
    m = 1
    for (row in 1:dims[1]){
      for (col in 1:dims[2]){
        value <- my.data[study, my.data.col.no.swapped[m]]
        m <- m + 1
        if (is.na(value)){
          next
        }
        value <- as.character(value)
        if (value == "nm"){
          study.df.list[[study]][row, col] <- nm.placeholder
        } else if (value == ""){
          # values stay NA
        } else if (no.char.but.digit.inside(value)){
          study.df.list[[study]][row, col] <- as.double(value)
        } else if (char.inside(value)){
          if (flag.x.s.r == 2){
            if (grepl("mean r", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean r.*", "", value))
                  # extracts anything before " mean r" as double                 
            } else if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if (flag.x.s.r == 1){
            if (grepl("mean s", value)){
              study.df.list[[study]][row, col] <- as.double(sub(" mean s.*", "", value))
            } else if (grepl("mean r", value)){
              # print(sprintf("mean r in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
            } else {
              study.df.list[[study]][row, col] <- value
            }
          } else if ((flag.x.s.r == 0) & !(grepl("mean r", value) | grepl("mean s", value))){
            if (grepl("mean", value)){
              # print(sprintf("mean only in %c, row %c, col %c", my.df[,"Study.ID"][study.no], dimname.list[1,row], dimname.list[2,col]))
              cat("value ", value, " not added")
              cat("\n")
              next
            }
            study.df.list[[study]][row, col] <- value
          } else {
            print("Unnown case appeared in:")
            print(my.df[study.no, "Study.ID"])
            cat("my.data.col.no: ", my.data.col.no.swapped[m])
            cat(sprintf("row %d, col %d", row, col))
            print(value)
            cat("\n")
          }
        } else {
          print("Unnown case appeared in:")
          print(my.df[study.no, "Study.ID"])
          cat("my.data.col.no: ", my.data.col.no.swapped[m])
          cat(sprintf("row %d, col %d\n\n", row, col))
        }
      }
    }
  }
  study.df.list
}


### Arrays
#### means.SDs


In [ ]:
dims <- c(4, 3, 4, 7, 2, study.no)

dimname.list <- list(c("Intervention.1", "Intervention.2", "Intervention.3", "Control"),
                     c("Mean", "SD", "n"),
                     c("T0", "T1", "T2", "T3"),
                     c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7"),
                     c("Scale.1", "Scale.2"),
                     my.df[,"Study.ID"])

start <- "Intervention.1.Mean.O1T0"
end <- "Control.or.Intervention.4.n.in.case.of.period.O7T3"

means.SDs.array <- clean.and.shape.data.to.array(start, end, dims, dimname.list) 


In [ ]:
# use my.pop.char.array[,,"T0","Johnson-Waddell 2018"] as test for "", NA, "nm", "double"
#      [,1]  [,2] [,3] [,4]  [,5]
# [1,] "197" NA   NA   "197" 394 
# [2,] ""    ""   NA   ""    "nm"
# [3,] ""    NA   NA   ""    "nm"
# [4,] ""    NA   NA   ""    "" 
# make flag.x.s.r work --> done


In [ ]:
means.SDs.array["Control","Mean","T1","Outcome.1","Janowiak 1994"]




#### Results Quantitative


In [ ]:
dims <- c(7, 3, 3, 6, 2, study.no)

dimname.list <- list(c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7"),
                     c("P.Value", "Effect.Size", "Effect.Size.CI"),
                     c("T1", "T2", "T3"),
                     c("Int1.X.Int2", "Int1.X.Int3", "Int1.X.Int4", "Int2.X.Int3", "Int2.X.Int4", "Int3.X.Int4"),
                     c("Scale.1", "Scale.2"),
                     my.df[,"Study.ID"]
                     )

start <- "Outcome.1.P.Value.1x2T1"
end <- "Outcome.7.Confidence.Interval.of.Effect.Size.3x4T3"

results.quantitative.array <- clean.and.shape.data.to.array(start, end, dims, dimname.list)


### Data frame lists
#### Intervention Comparison


In [ ]:
dims <- c(4, 7)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Intervention.1", "Intervention.2", "Intervention.3", "Control"),
                     c("Name", "Short.Description", "Delivery.Mode", "Meditation.App", "Sessions.Duration.in.minutes",
                       "Frequency.in.times.per.week", "Total.Duration.in.Days")
                     )
start <- "Name.Intervention.1"
end <- "Total.Duration.of.Intervention.in.Days.Control.or.Intervention.4"

intervention.comparisons.df.list <- clean.data.to.df.list.swap(start, end, dims, list.names, dimname.list)
intervention.comparisons.df.list



#### Dates of Measuring Time Points


In [ ]:
dims <- c(4, 1)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Time.Point.0", "Time.Point.1", "Time.Point.2", "Time.Point.3"),
                     c("Date")
                     )
start <- "Time.Point.0.Date"
end <- "Time.Point.3.Date"

dates.measuring.time.points.df.list <- clean.data.to.df.list(start, end, dims, list.names, dimname.list)
dates.measuring.time.points.df.list



#### Between-Measuring Time Points Duration


In [ ]:
dims <- c(3, 1)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Time.Point.0...1", "Time.Point.0...2", "Time.Point.0...3"),
                     c("Duration.in.Days")
                     )
start <- "Time.Point.0...1.Duration.in.Days"
end <- "Time.Point.0...3.Duration.in.Days"

between.T.duration.df.list <- clean.data.to.df.list(start, end, dims, list.names, dimname.list)
between.T.duration.df.list



#### Definition of Outcomes


In [ ]:
dims <- c(7, 1)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7"),
                     c("Definition")
                     )
start <- "Outcome.1.Definition"
end <- "Outcome.7.Definition"

outcome.definitions.df.list <- clean.data.to.df.list(start, end, dims, list.names, dimname.list)
outcome.definitions.df.list



#### Measures of Outcomes


In [ ]:
dims <- c(7, 3)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7"),
                     c("Measures.Name", "Start.and.End.Point", "High.or.low.means.resilient")
                     )
start <- "Outcome.1.Scale.s.or.other.Measure.s.Name"
end <- "Outcome.7.High.or.low.means.resilient"

outcome.measures.df.list <- clean.data.to.df.list(start, end, dims, list.names, dimname.list)
outcome.measures.df.list



#### Qualitative Results of Data Analyses


In [ ]:
dims <- c(7, 3)

list.names <- my.df[,"Study.ID"]

dimname.list <- list(c("Key.Theme.1", "Key.Theme.2", "Key.Theme.3", "Key.Theme.4", "Key.Theme.5", "Key.Theme.6", "Key.Theme.7"),
                     c("Name.of.Key.Theme", "Description.of.Key.Theme", "Results")
                     )
start <- "Key.Theme.1.Name.of.Key.Theme"
end <- "Key.Theme.7.Results"

results.qualitative.df.list <- clean.data.to.df.list(start, end, dims, list.names, dimname.list)
results.qualitative.df.list


# Notes
- Different Scales
    - 2 Array-Dims with same values
    - Results in Dim "Scale.2" only if 2nd scale is present, else: NA


# ToDo
- chekc function for array beeing correct for dim == 6 --> type respective code fpr dim ==6
- Check data for if "mean" (only) is attended to values instead of "mean r" or "mean s"
  - in df function - make warning work: 'print(sprintf("mean r in %c, row %c, col %c", my.df[,"Study.ID"][study.no],
    dimname.list[1,row], dimname.list[2,col]))'
- find solution for value: '3.79 during intervention, 2.04 during follow-up mean'
- Handle NA Warnings for
  - Measure of Outcomes
  - Results Quantitative
- check all functions for beeing correct
- put total data into a list


# Array Dims and Dimnames
Dates of Measuring Time Points
	4 x 1
	c("Time.Point.0", "Time.Point.1", "Time.Point.2", "Time.Point.3")
	c("Date")
	Time.Point.0.Date:Time.Point.3.Date
	
Between-Measuring Time Points' Duration
	3 x 1
	c("Time.Point.0...1", "Time.Point.0...2", "Time.Point.0...3")
	c("Duration.in.Days")
	Time.Point.0...1:Duration.in.Days

Intervention and Comparisons
	7 x 4
	c("Name", "Short.Description", "Delivery.Mode", "Meditation.App", "Sessions.Duration.in.minutes", Frequency.in.times.per.week", "Total.Duration.in.Days")
	c("Intervention.1", "Intervention.2", "Intervention.3", "Control")
	Name.Intervention 1:Total.Duration.of.Intervention.in.Days.Control.or.Intervention.4

Definition of Outcomes
	7 x 1
	c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7")
	c("Definition")
	Outcome.1:Definition

Measures of Outcomes
	7 x 3
	c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7")
	c("Measures.Name", "Start.and.End.Point", "High.or.low.means.resilient")
	Outcome.1.Scale.s.or.other.Measure.s.Name:Outcome.7.High.or.low.means.resilient
	
Means and SDs of Outcomes
	4 x 3 x 4 x 7
	c("Intervention.1", "Intervention.2", "Intervention.3", "Control"
	c("Mean", "SD", "n")
	c("T0", "T1", "T2", "T3")
	c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7")
	Intervention.1.Mean.O1T0:Control.or.Intervention.4.n.in.case.of.period.O7T3

Quantitative Results of Data Analyses
	7 x 3 x 3 x 6
	c("Outcome.1", "Outcome.2", "Outcome.3", "Outcome.4", "Outcome.5", "Outcome.6", "Outcome.7")
	c("P.Value", "Effect.Size", "Effect.Size.CI")
	c("T1", "T2", "T3")
	c("Int1.X.Int2", "Int1.X.Int3", "Int1.X.Int4", "Int2.X.Int3", "Int2.X.Int4", "Int3.X.Int4")
	Outcome.1.P.Value.1x2T1:Outcome.7.Confidence.Interval.of.Effect.Size.3x4T3

Qualitative Results of Data Analyses
	7 x 3
	c("Key.Theme.1", "Key.Theme.2", "Key.Theme.4", "Key.Theme.5", "Key.Theme.6", "Key.Theme.7")
	c("Name.of.Key.Theme", "Description.of.Key.Theme", "Results")
	Key.Theme.1:Results

# TEST - Get Studies investigating stress


In [ ]:
my.df.stress <- my.df[my.df$Name.of.Outcome.1 == "Stress" |
                      my.df$Name.of.Outcome.2 == "Stress" |
                      my.df$Name.of.Outcome.3 == "Stress" |
                      my.df$Name.of.Outcome.4 == "Stress" |
                      my.df$Name.of.Outcome.5 == "Stress" |
                      my.df$Name.of.Outcome.6 == "Stress" |
                      my.df$Name.of.Outcome.7 == "Stress",]


In [ ]:
subset(my.df.stress, select = c(Name.of.Outcome.1:Name.of.Outcome.7))



In [ ]:
my.df.stress.id.otcm <- subset(my.df.stress, select = c(Study.ID, Name.of.Outcome.1:Name.of.Outcome.7))
my.df.stress.id.otcm


# TEST - Convert data frame to array and insert array into it
## Create df


In [ ]:
test.name.1 <- 1
test.name.2 <- 2
test.name.3 <- 3
test.name.4 <- 4


test.df <- data.frame(test.name.1,
                      test.name.2,
                      test.name.3,
                      test.name.4
                      )


In [ ]:
test.df




## Convert df to array


In [ ]:
arr1 <- array(data = c(unlist(test.df)),
              dim = c(1, 4),
              dimnames = list("Jacob 2022",
                              colnames(test.df)
                              )
)



## Split array in to parts


In [ ]:
cut.range.1 <- which(colnames(arr1) == "test.name.1"):which(colnames(arr1) == "test.name.2")



In [ ]:
arr1.1 <- array(data = arr1[, cut.range.1],
                dim = c(1,length(cut.range.1)),
                dimnames = list(rownames(arr1),
                                colnames(arr1)[cut.range.1]
                                )
                )
arr1.1


In [ ]:
cut.range.2 <- which(colnames(arr1) == "test.name.3"):which(colnames(arr1) == "test.name.4")



In [ ]:
arr1.2 <- array(data = arr1[, cut.range.2],
                dim = c(1,length(cut.range.2)),
                dimnames = list(rownames(arr1),
                                colnames(arr1)[cut.range.2]
                                )
                )
arr1.2



## Create array that will get inserted


In [ ]:
arr0 <- array(data = 1:4,
              dim = c(2, 2),
              dimnames = list(c("Row.1", "Row.2"),
                              c("Col.1", "Col2"))
              )



## Create new array out of these 3


In [ ]:
arr2 <- array(data = c(arr1.1, arr1.2),
              dim = c(dim(arr1.2)[1],
                      dim(arr1.1)[2] + dim(arr1.2)[2]
                      ),
              dimnames = list(rownames(arr1.1),
                              c(colnames(arr1.1),
                                colnames(arr1.2)
                                )
                              )
              )

# possible to combine arrays of different dimension in on array?


In [ ]:
list.1 <- list(arr1.1, arr0, arr1.2)
names(list.1) <- c("arr1.1", "arr0", "arr1.2")
list.1


In [ ]:
arr3 <- array(data = c(list.1, list.1))
arr3
